# Peloton Connect

### Reference doc
https://hackernoon.com/i-used-python-to-analyze-my-peloton-workout-stats-with-real-time-updates-5j5q31rs
https://www.shipyardapp.com/blog/how-to-use-peloton-data/

### Inspiration

* I am an avid peloton user
* I've been training through powerzone classes
* The basic metrics I typically take a look at is:
   *   total lengh of class
   *   total output
   *   leaderboard ranking


In [2]:
import requests
import json
import pandas as pd
import os

peloton_username = os.environ.get('peloton_user_name') 
peloton_password = os.environ.get('peloton_password') 

s = requests.Session()
base_url = 'https://api.onepeloton.com'
payload = {'username_or_email': peloton_username, 'password': peloton_password}
s.post(base_url + '/auth/login', json=payload)
userID = s.get(base_url + '/api/me').json()['id']
workouts = json.loads(json.dumps(s.get(base_url + '/api/user/' + userID + '/workouts?limit=1000').json()).replace('null', '""'))

In [3]:
def save_extracted_workout_data(data):
  data_json = json.dumps(data)  
  with open('workout_data.json', 'w') as file:
        file.write(data_json)

def load_extracted_workout_data():
    with open('workout_data.json', 'r') as file:
        workout_data_json = file.read()
    # Convert JSON string to workout_data dictionary
    workout_data = json.loads(workout_data_json)
    return(workout_data)

def get_workout_data():
    workout_keys = list(workout_data.keys())
    workouts_summary = []
    
    for key in workout_keys:
        workout_id = key
        workout =  workout_data[workout_id]
        workout_summary = workout['summary']
        workout_performance = workout['performance']
        summary = workout_performance['summaries']
        avg_summary = workout_performance['average_summaries']    

        workout_dict = {
            'workout_id': workout_id,
            'title': workout_summary['ride'].get('title'),
            'category': workout_summary['fitness_discipline'],
            'description': workout_summary['ride'].get('description'),
            'instructor_id': workout_summary['ride'].get('instructor_id'),
            'duration': workout_summary['ride'].get('duration'),
            'workout_start_ts': workout_summary['created_at'],
            'workout_end_ts': workout_summary['end_time'],
            'achievements': workout_summary['achievement_templates'],
            'leaderboard_rank': workout_summary['leaderboard_rank'],
            'total_leaderboard_users': workout_summary['total_leaderboard_users'],
            'total_output': next((d['value'] for d in summary if d.get("display_name") == 'Total Output'), None),
            'total_distance': next((d['value'] for d in summary if d.get("display_name") == 'Distance'), None),
            'total_calories': next((d['value'] for d in summary if d.get("display_name") == 'Calories'), None),
            'average_output': next((d['value'] for d in avg_summary if d.get("display_name") == 'Avg Output'), None),
            'average_cadence': next((d['value'] for d in avg_summary if d.get("display_name") == 'Avg Cadence'), None),
            'average_resistance': next((d['value'] for d in avg_summary if d.get("display_name") == 'Avg Resistance'), None),
            'average_speed': next((d['value'] for d in avg_summary if d.get("display_name") == 'Avg Speed'), None)
        }
               
        workouts_summary.append(workout_dict)
        
    return workouts_summary

**NOTE** 

The call below will stop at 1 min mark. I am not sure if this is due to the limit on DataCamp Workspace or Peloton.
I should eventually write a function to:

  * (1) get all workout key, and upsert it to database.
  * (2) get summary data for workouts, if it does not exist in the db. Upsert.
  * (3) get performance data for workout, if it does not exist in the db. Upsert


In [4]:
# construct workout data
workout_data = {}
for workout in workouts['data']:
    workout_id = workout['id']
    workout_data[workout_id] = {}
    
    # api call to get summary and performance
    workout_summary = json.loads(json.dumps(s.get(base_url + '/api/workout/' + workout_id).json()).replace('null', '""'))
    workout_performance = json.loads(json.dumps(s.get(base_url + '/api/workout/' + workout_id + '/performance_graph?every_n=1000').json()).replace('null', '""'))
    workout_data[workout_id]['summary'] = workout_summary   
    workout_data[workout_id]['performance'] = workout_performance

In [5]:
save_extracted_workout_data(workout_data)

In [6]:
workout_output = get_workout_data()

In [7]:
workout_output_df = pd.DataFrame(workout_output)

workout_output_df['workout_start_ts'] = pd.to_datetime(workout_output_df['workout_start_ts'], unit='s')
workout_output_df['workout_end_ts'] = pd.to_datetime(workout_output_df['workout_end_ts'], unit='s')
workout_output_df['duration'] = workout_output_df['duration'] / 60

In [41]:
workout_output_df

,workout_id,title,category,description,instructor_id,duration,workout_start_ts,workout_end_ts,achievements,leaderboard_rank,total_leaderboard_users,total_output,total_distance,total_calories,average_output,average_cadence,average_resistance,average_speed
0,c4bb12b1cfca4c48897d7c2e5edb15cf,10 min Full Body Stretch,stretching,"That's a wrap on week 2, so celebrate that you...",7f3de5e78bb44d8591a0f77f760478c3,10.000000,2024-02-05 00:46:24,2024-02-05 00:56:53,[],,0,NaN,NaN,51.0,NaN,NaN,NaN,NaN
1,19546ac0bcbc49fd80697634e0e0a1cc,20 min Full Body Strength,strength,We're nearly at the half-way mark — look how f...,7f3de5e78bb44d8591a0f77f760478c3,20.000000,2024-02-05 00:24:06,2024-02-05 00:45:06,"[{'id': 'df98e7119e4b478ea02494f22c004fe6', 'n...",,0,NaN,NaN,203.0,NaN,NaN,NaN,NaN
2,4388299f871f4f9dac501acd7974e194,5 min Lower Body Stretch,stretching,Unwind in this 5-minute lower-body stretch. St...,7f3de5e78bb44d8591a0f77f760478c3,5.000000,2024-02-04 17:30:06,2024-02-04 17:35:36,[],,0,NaN,NaN,25.0,NaN,NaN,NaN,NaN
3,44a729a94b9b44ad9b1a83b6a46ebf99,20 min Lower Body & Core Strength,strength,Today's focus is lower body & core — continue ...,7f3de5e78bb44d8591a0f77f760478c3,20.000000,2024-02-04 17:07:56,2024-02-04 17:28:56,"[{'id': '5298b832e2274ad59cf8857240440fb2', 'n...",,0,NaN,NaN,203.0,NaN,NaN,NaN,NaN
4,ef705a070ce247d4a08f6a97bb007b5f,10 min Lower Body Stretch,stretching,Recovery starts here! Your muscles will thank ...,c9fa21c2004c4544a7c35c28a6196c77,10.000000,2024-02-04 01:03:57,2024-02-04 01:13:58,"[{'id': '1b0f7ba0b9e945e88c93792484995c00', 'n...",,0,NaN,NaN,56.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,50c3ca2db35741e997bf8375f84b29b9,10 min Strength Roll Call: Core,strength,Thursdays are all about the core. Join us on t...,7f3de5e78bb44d8591a0f77f760478c3,10.000000,2023-12-25 15:17:23,2023-12-25 15:28:23,"[{'id': '657e50c747d6458480f1ba6a0fa94c6a', 'n...",,0,NaN,NaN,101.0,NaN,NaN,NaN,NaN
96,6141a0d8ae974713b9ac2fe5f3059e78,5 min Post-Ride Stretch,stretching,Join Bradley for this 5-minute stretch complem...,01f636dc54a145239c4348e1736684ee,5.000000,2023-12-24 20:11:29,2023-12-24 20:16:30,"[{'id': '657e50c747d6458480f1ba6a0fa94c6a', 'n...",,0,NaN,NaN,25.0,NaN,NaN,NaN,NaN
97,72b6a8b62a314b96861e9b07f353b481,30 min 30 Seconds to Mars,cycling,None,None,30.183333,2023-12-24 19:39:01,2023-12-24 20:09:12,"[{'id': '4036702255e84f26bee944331ef92310', 'n...",,0,439.0,11.4,1046.0,243.0,74.0,52.0,22.8
98,ad5a64e21f8a4578b5ec72747084c632,30 min Legs & Shoulders,strength,Join us on the mat for this upper- and lower-b...,7f3de5e78bb44d8591a0f77f760478c3,30.000000,2023-12-24 02:08:03,2023-12-24 02:39:33,"[{'id': '22714437c08e478f8b67e0d1364f415b', 'n...",,0,NaN,NaN,283.0,NaN,NaN,NaN,NaN


### 45 min endurance workouts

In [8]:
df_45min_endurance = workout_output_df[\
                                       (workout_output_df['category'] == 'cycling') &\
                                       (workout_output_df['duration'] == 45)]

In [9]:
df_45min_endurance['leaderboard_percentile'] = (df_45min_endurance['leaderboard_rank'] / df_45min_endurance['total_leaderboard_users']) * 100

In [10]:
df_45min_endurance['achievements_list'] = df_45min_endurance['achievements'].apply(lambda x: [achievement['name'] for achievement in x])

In [11]:
df_45min_endurance

,workout_id,title,category,description,instructor_id,duration,workout_start_ts,workout_end_ts,achievements,leaderboard_rank,total_leaderboard_users,total_output,total_distance,total_calories,average_output,average_cadence,average_resistance,average_speed,leaderboard_percentile,achievements_list
13,647f21d318d34189a29d3a3a6c952910,45 min Power Zone Endurance Ride,cycling,11 min warm-up followed by four 6 min Z3 inter...,7f3de5e78bb44d8591a0f77f760478c3,45.0,2024-04-23 00:50:07,2024-04-23 01:36:07,"[{'id': '7a68d49d95ce4918b7408b26f91d9eac', 'n...",1772,167076,653.0,17.12,680,242.0,102.0,46.0,22.8,1.060595,[Flock]
19,906b4601729c4bfb921381b2947cfcf2,45 min Power Zone Ride,cycling,12 min warm-up followed by three blocks (8/12/...,1e59e949a19341539214a4a13ea7ff01,45.0,2024-04-21 00:46:11,2024-04-21 01:32:11,"[{'id': '7a68d49d95ce4918b7408b26f91d9eac', 'n...",1579,185716,699.0,17.51,664,259.0,103.0,47.0,23.3,0.850223,[Flock]
31,0b15525e3e724ef0ace5230896da8c31,45 min Power Zone Endurance Ride,cycling,12 min warm-up followed by five Z3 intervals (...,05735e106f0747d2a112d32678be8afd,45.0,2024-04-15 20:18:57,2024-04-15 21:04:57,"[{'id': '5298b832e2274ad59cf8857240440fb2', 'n...",1821,210657,661.0,17.19,767,245.0,103.0,46.0,22.9,0.864438,"[3-Day Streak, Flock]"
35,39c29978f450482195ca023b38e39246,45 min Power Zone Endurance Ride,cycling,13 min warm-up followed by five Z3 intervals (...,304389e2bfe44830854e071bffc137c9,45.0,2024-04-13 21:00:38,2024-04-13 21:46:38,"[{'id': '7a68d49d95ce4918b7408b26f91d9eac', 'n...",2674,253334,628.0,16.87,601,233.0,100.0,46.0,22.5,1.055524,"[Flock, April Cycling Challenge]"
37,74304baee56a415b9bb49b331466d092,45 min Power Zone Endurance Ride,cycling,Train smart with 7 zones of output customized ...,304389e2bfe44830854e071bffc137c9,45.0,2024-04-12 00:49:26,2024-04-12 01:35:26,"[{'id': '3a9ea8169d17455c86b9f52b1011e57b', 'n...",802,82034,738.0,17.92,711,273.0,93.0,52.0,23.9,0.977643,[Squad]
42,b0245c4e94e14c55aa831650c2e53a37,45 min Power Zone Endurance Rock Ride,cycling,Train smart with 7 zones of output customized ...,7f3de5e78bb44d8591a0f77f760478c3,45.0,2024-04-08 01:15:22,2024-04-08 02:01:23,"[{'id': '3a9ea8169d17455c86b9f52b1011e57b', 'n...",1010,61818,703.0,17.60,679,260.0,97.0,50.0,23.5,1.633828,[Squad]
53,0d6eaef4090b40be8107d274470527a3,45 min Power Zone Endurance Ride,cycling,Train smart with 7 zones of output customized ...,5a19bfe66e644a2fa3e6387a91ebc5ce,45.0,2024-04-01 18:02:06,2024-04-01 18:48:06,"[{'id': '529f22af30ba45a0a501b8542ad3b99c', 'n...",189,27292,738.0,17.94,682,273.0,96.0,51.0,23.9,0.692511,"[60-Day Streak, Pack]"
57,d9e9ccc095444c4fb439566521427841,45 min Power Zone Ride,cycling,Train smart with 7 zones of output customized ...,5a19bfe66e644a2fa3e6387a91ebc5ce,45.0,2024-03-30 23:58:02,2024-03-31 00:44:02,"[{'id': 'f0bdc95051b64c5bbd296e20d3fecb03', 'n...",122,14167,731.0,17.72,637,271.0,90.0,53.0,23.6,0.861156,[Pack]
61,3460db1b388e4bef86bcacb86e477691,45 min Power Zone Ride,cycling,Train smart with 7 zones of output customized ...,5a19bfe66e644a2fa3e6387a91ebc5ce,45.0,2024-03-28 00:22:22,2024-03-28 00:50:30,"[{'id': 'f0bdc95051b64c5bbd296e20d3fecb03', 'n...",5117,14167,389.0,10.14,376,239.0,89.0,51.0,22.4,36.11915,[Pack]
65,90a4ae2768e349db81450d309c458948,45 min Power Zone Endurance EDM Ride,cycling,Train smart with 7 zones of output customized ...,3ff679ebbd324c83a8ab6cfa6bb4be37,45.0,2024-03-26 00:19:49,2024-03-26 01:05:49,"[{'id': 'f0bdc95051b64c5bbd296e20d3fecb03', 'n...",514,36189,710.0,17.71,665,263.0,94.0,51.0,23.6,1.420321,[Pack]


### Stretching

In [12]:
workout_output_df.category.unique()

array(['strength', 'stretching', 'cycling', 'caesar', 'walking'],
      dtype=object)

In [13]:
df_stretching = workout_output_df[(workout_output_df['category'] == 'stretching')]
df_stretching['achievements_list'] = df_stretching['achievements'].apply(lambda x: [achievement['name'] for achievement in x])

In [14]:
df_stretching

,workout_id,title,category,description,instructor_id,duration,workout_start_ts,workout_end_ts,achievements,leaderboard_rank,total_leaderboard_users,total_output,total_distance,total_calories,average_output,average_cadence,average_resistance,average_speed,achievements_list
2,bec49ede0d5441908890f30204e28591,10 min Stretch,stretching,Improve performance by stretching after your C...,c9fa21c2004c4544a7c35c28a6196c77,10.0,2024-04-27 00:44:03,2024-04-27 00:54:04,"[{'id': '4dafabb0092b4870bff8ef393dc1acbe', 'n...",,0,NaN,NaN,51,NaN,NaN,NaN,NaN,"[10-Day Streak, Dynamic Duo]"
8,c3c28b9aebe54bed8310a17b7113efed,10 min Evening Stretch,stretching,Recovery starts here! Whether you take this fu...,a606b2c39c194bcc80f9a541b97b4537,10.0,2024-04-24 01:20:38,2024-04-24 01:30:39,"[{'id': '657e50c747d6458480f1ba6a0fa94c6a', 'n...",,0,NaN,NaN,54,NaN,NaN,NaN,NaN,[Dynamic Duo]
11,510feef4a6c24a4ba8661c01e3d2cbbb,10 min Stretch,stretching,Improve performance by stretching after your s...,c9fa21c2004c4544a7c35c28a6196c77,10.0,2024-04-24 00:33:31,2024-04-24 00:43:32,"[{'id': 'ca11fdd59d6342809084532531f18ed9', 'n...",,0,NaN,NaN,51,NaN,NaN,NaN,NaN,"[7-Day Streak, Dynamic Duo, April Activity Cha..."
16,83649cc788d941199c2baac158da9f5f,10 min Stretch,stretching,Improve performance by stretching after your C...,c9fa21c2004c4544a7c35c28a6196c77,10.0,2024-04-21 01:51:56,2024-04-21 02:01:57,"[{'id': '657e50c747d6458480f1ba6a0fa94c6a', 'n...",,0,NaN,NaN,63,NaN,NaN,NaN,NaN,[Dynamic Duo]
25,8a07d77ace3b430fbdc3866bff6ef5eb,10 min Stretch,stretching,Improve performance by stretching after your C...,c9fa21c2004c4544a7c35c28a6196c77,10.0,2024-04-18 01:30:50,2024-04-18 01:40:51,"[{'id': 'c8d261b05c5f403b83986a2c43654d1d', 'n...",,0,NaN,NaN,62,NaN,NaN,NaN,NaN,"[350 Stretching Workouts, Dynamic Duo]"
29,8cd57de8853d409586321134144b87ff,5 min Post-Ride Stretch,stretching,Join Alex for this 5-minute stretch complement...,2e57092bee334c8c8dcb9fe16ba5308c,5.0,2024-04-15 21:13:53,2024-04-15 21:18:54,[],,0,NaN,NaN,28,NaN,NaN,NaN,NaN,[]
33,1bd3da1f9f42406393505905b162d91e,5 min Pre-Run Warm Up,stretching,Get your mind and muscles ready to run in this...,c9fa21c2004c4544a7c35c28a6196c77,5.0,2024-04-13 21:55:32,2024-04-13 22:00:33,"[{'id': '657e50c747d6458480f1ba6a0fa94c6a', 'n...",,0,NaN,NaN,30,NaN,NaN,NaN,NaN,[Dynamic Duo]
40,5e966e72309a4568b9af128ac990c86c,10 min Evening Stretch,stretching,Recovery starts here! Whether you take this fu...,a606b2c39c194bcc80f9a541b97b4537,10.0,2024-04-08 02:09:45,2024-04-08 02:19:47,[],,0,NaN,NaN,61,NaN,NaN,NaN,NaN,[]
48,5d0c4ceb16ca4677a2b51b21a1a0458b,10 min Lower Body Stretch,stretching,Recovery starts here! Your muscles will thank ...,c9fa21c2004c4544a7c35c28a6196c77,10.0,2024-04-03 01:17:41,2024-04-03 01:27:41,"[{'id': '1b0f7ba0b9e945e88c93792484995c00', 'n...",,0,NaN,NaN,51,NaN,NaN,NaN,NaN,[Three's Company]
51,0a199383b4a3499aa952fb08da9cb33d,10 min Lower Body Stretch,stretching,Recovery starts here! Your muscles will thank ...,a606b2c39c194bcc80f9a541b97b4537,10.0,2024-04-01 19:00:07,2024-04-01 19:10:08,"[{'id': '1b0f7ba0b9e945e88c93792484995c00', 'n...",,0,NaN,NaN,51,NaN,NaN,NaN,NaN,[Three's Company]
